In [6]:
# Importing libraries
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
import seaborn as sns
from collections import Counter
from numpy.random import randn
from datetime import datetime

from geopy.distance import vincenty
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from collections import Counter
from scipy import stats
from dateutil import parser
# import eli5
# from eli5.sklearn import PermutationImportance
# from skopt.space import Real
import warnings
warnings.filterwarnings("ignore")





import matplotlib.pyplot as plt
%matplotlib inline





# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

C:\Users\mvesk\Anaconda3\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
C:\Users\mvesk\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
negReviews = pd.read_csv('Review_ASCII_Negative.csv')
df= negReviews

In [8]:
# Seperate NLP features
nlp_feats = ['comments']
corpus = df[nlp_feats]
df = df.drop(nlp_feats, axis = 1)
print("Dataset has {} rows, {} columns.".format(*df.shape))

Dataset has 6098 rows, 19 columns.


In [9]:
import nltk
#nltk.data.path.append("\Users\mvesk\Project3\all_listings.csv")


from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim.models.ldamodel import LdaModel

import itertools
from collections import Counter
from collections import defaultdict

import json
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [16]:
def preprocess_text(corpus):
    """Takes a corpus in list format and applies basic preprocessing steps of word tokenization,
     removing of english stop words, lower case and lemmatization."""
    processed_corpus = []
    english_words = set(nltk.corpus.words.words())
    english_stopwords = set(stopwords.words('english'))
#     customize_stop_words = [
#     'kilda', 'u','mins','didn','myk','emma','peter','de','st','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30',
# 	'41','42','43','44','45','46','47','48','49','50','51','52','53','54','55','56','57','58','59','60','61','62','63','64','65','66','67','68','69','70','.']
#     english_stopwords.extend(customize_stop_words)
    wordnet_lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'[\w|!]+')
    for row in corpus:
        word_tokens = tokenizer.tokenize(row)
        word_tokens_lower = [t.lower() for t in word_tokens]
        word_tokens_lower_english = [t for t in word_tokens_lower if t in english_words or not t.isalpha()]
        word_tokens_no_stops = [t for t in word_tokens_lower_english if not t in english_stopwords]
        word_tokens_no_stops_lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in word_tokens_no_stops]
        processed_corpus.append(word_tokens_no_stops_lemmatized)
    return processed_corpus

In [17]:
def nlp_model_pipeline(processed_corpus):
    """Takes processed corpus and produce dictionary, doc_term_matrix and LDA model"""
    # Creates the term dictionary (every unique term in corpus is assigned an index)
    dictionary = Dictionary(processed_corpus)
    # Convert corpus into Document Term Matrix using dictionary prepared above
    doc_term_matrix = [dictionary.doc2bow(listing) for listing in processed_corpus]    
    return dictionary, doc_term_matrix

In [18]:
def LDA_topic_modelling(doc_term_matrix, dictionary, num_topics=3, passes=2):
    # Create an object for LDA model and train it on Document-Term-Matrix
    LDA = LdaModel
    ldamodel = LDA(doc_term_matrix, num_topics=num_topics, id2word = dictionary, passes=passes)
    return ldamodel

In [19]:
	def add_topics_to_df(ldamodel, doc_term_matrix, df, new_col, num_topics):
    # Convert into Per-document topic probability matrix:
    docTopicProbMat = ldamodel[doc_term_matrix]
    docTopicProbDf = pd.DataFrame(index=df.index, columns=range(0, num_topics))
    for i, doc in enumerate(docTopicProbMat):
        for topic in doc:
            docTopicProbDf.iloc[i, topic[0]] = topic[1]
    docTopicProbDf[new_col] = docTopicProbDf.idxmax(axis=1)
    df_topics = docTopicProbDf[new_col]
    # Merge with df
    df_new = pd.concat([df, df_topics], axis=1)
    return df_new

In [20]:
corpus.head()

,comments
0,.
1,Top
2,43748
3,The
4,S


In [21]:
%%time
corpus_comments= corpus['comments'].astype(str)
processed_corpus_comments = preprocess_text(corpus_comments)
dictionary_comments, doc_term_matrix_comments = nlp_model_pipeline(processed_corpus_comments)


Wall time: 3.23 s


In [24]:
%%time
ldamodel_comments = LDA_topic_modelling(doc_term_matrix_comments, dictionary_comments, num_topics=10, passes=20)

Wall time: 37.7 s


In [25]:
%%time
negative=pyLDAvis.gensim.prepare(ldamodel_comments, doc_term_matrix_comments, dictionary_comments)

Wall time: 1.97 s


In [26]:
negative

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.312124  0.068125       1        1  26.183161
1      0.228400  0.150756       2        1  22.231487
9     -0.278160  0.031091       3        1  20.556786
8     -0.113972  0.160134       4        1   6.753747
5      0.031338  0.133709       5        1   5.854707
4      0.150949 -0.012619       6        1   5.323532
3      0.216468  0.115820       7        1   4.710546
6      0.014385 -0.270615       8        1   4.057587
2      0.017689 -0.263899       9        1   2.306798
7      0.045028 -0.112503      10        1   2.021646, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
191   Default  2850.000000        arrival  2850.000000  30.0000  30.0000
192   Default  2404.000000    reservation  2404.000000  29.0000  29.0000
344   Default  2394.000000        posting  2394.000000  28.0000  28.0000
146   Default  3195.000000            day  3195.000000  27.0000  27.0000
81    Default  2368.000000           host  2368.000000  26.0000  26.0000
63    Default   688.000000              u   688.000000  25.0000  25.0000
269   Default   523.000000         street   523.000000  24.0000  24.0000
86    Default   768.000000          would   768.000000  23.0000  23.0000
518   Default   455.000000       bathroom   455.000000  22.0000  22.0000
182   Default   553.000000          night   553.000000  21.0000  21.0000
598   Default   885.000000        however   885.000000  20.0000  20.0000
214   Default   442.000000           left   442.000000  19.0000  19.0000
114   Default   409.000000          clean   409.000000  18.0000  18.0000
168   Default   399.000000            get   399.000000  17.0000  17.0000
238   Default   263.000000        bedroom   263.000000  16.0000  16.0000
256   Default   352.000000              2   352.000000  15.0000  15.0000
508   Default   313.000000         shower   313.000000  14.0000  14.0000
27    Default   639.000000           room   639.000000  13.0000  13.0000
109   Default   350.000000           good   350.000000  12.0000  12.0000
299   Default   262.000000        contact   262.000000  11.0000  11.0000
72    Default  1030.000000      apartment  1030.000000  10.0000  10.0000
5     Default   217.000000              1   217.000000   9.0000   9.0000
303   Default   238.000000        without   238.000000   8.0000   8.0000
216   Default   509.000000           time   509.000000   7.0000   7.0000
503   Default   256.000000           like   256.000000   6.0000   6.0000
510   Default   245.000000        kitchen   245.000000   5.0000   5.0000
199   Default   482.000000       although   482.000000   4.0000   4.0000
241   Default   312.000000          could   312.000000   3.0000   3.0000
309   Default   224.000000          check   224.000000   2.0000   2.0000
287   Default   336.000000              3   336.000000   1.0000   1.0000
19     Topic1   277.456390          dirty   277.856110   1.3386  -4.3537
218    Topic1   171.356018          quite   171.755814   1.3377  -4.8356
325    Topic1   147.681946           made   148.081787   1.3373  -4.9843
75     Topic1   114.147995           late   114.547920   1.3366  -5.2419
499    Topic1   103.978462         couple   104.378304   1.3362  -5.3352
323    Topic1    96.287422           feel    96.687157   1.3359  -5.4120
217    Topic1    94.023956           easy    94.423859   1.3358  -5.4358
164    Topic1    92.893929           walk    93.293747   1.3358  -5.4479
44     Topic1    90.438904          noisy    90.838631   1.3356  -5.4747
669    Topic1   137.385925           know   138.043076   1.3353  -5.0566
263    Topic1    74.782249          whole    75.182121   1.3347  -5.6648
1501   Topic1    62.348812          glass    62.765072   1.3334  -5.8466
76     Topic1    56.019794          sorry    56.419559   1.3329  -5.9537
2680   Topic1    54.4045

In [27]:

pyLDAvis.save_html(negative,'negative10_lda.html')